<a href="https://colab.research.google.com/github/manufabregas/Dataset-Asistencia-a-turnos-m-dicos/blob/main/TP%202/Pr%C3%A1ctico_2_An%C3%A1lisis_y_curaci%C3%B3n_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo y alcance
En esta segunda aproximación (que suele ir de la mano con el análisis exploratorio y visualización) se pretende empezar a transformar el dataset para los primeros experimentos de aprendedores en las siguientes materias.

En este laboratorio el primer gran objetivo va a ser lograr un único dataset con columnas nuevas, que condense toda la información importante para el posterior entrenamiento y aprendizaje que permitirá conocer si algunas de las variables incluidas en el mismo tienen influencia sobre la problemática que nos atañe.

## Lista de tareas

1.   Verificar la información brindada por el dataset a partir del práctico n°1. ¿Hay algo que les haya llamado la atención y consideran que puede estar mal tomado el dato, o sesgado? La inclusión del mismo, ¿podría generar problemas a futuro? Tomar una decisión por cada dato o información que consideren no fiable y justificar su respuesta.
2.   Verificar y asegurar que no existan datos duplicados.
3.   Verificar que no existan caracteres "raros" para los datos de las variables de especialidad (msp_name) y nombre de especialistas (doc_full_name).
4.   A partir de los valores faltantes reconocidos en el práctico N° 1 se deberán proponer dos soluciones posibles para trabajar con estos datos. En este punto deberán ejecutar ambas soluciones y fundamentar su aplicación. Además, deberán elegir una para obtener su dataset final, explicando su elección.
5.   Hay algunas variables que contienen datos de texto pero no están normalizados, como por ejemplo "msp_name", donde encontramos que ginecología puede estar escrita como "GINECOLOGÍA", "ginecologia", "ginecologa", entre otras variantes. Es importante aplicar alguna técnica para unificar estos datos. Se recomienda el uso de [regex](https://regexr.com/).
6.   Outliers: ¿Existen valores atípicos en las columnas del dataset? A partir del análisis del práctico n°1 aplicar una solución para manejar solo esos outliers.
7.   A partir de la exploración del práctico n°1, ¿qué variables nuevas se les ocurre que pueden generar y sean de importancia pensando en el problema de investigación planteado?. Se espera que puedan construir al menos 3 variables nuevas a partir de la información contenida en la base de datos (pueden usar las generadas en el práctico 1 pero es importante que se justifique por qué son interesantes de considerar respecto al problema).
8.   Reconocer las relaciones entre las distintas variables con la feature target del problema a resolver en el práctico 3.
9.   Si tuvieran que agregar datos que no son posibles de extraer a partir de la información que brinda el dataset, y que pueden suponer una gran influencia en nuestra variable objetivo, ¿Qué datos utilizarían? ¿Cómo plantearían su obtención? (teórico)
10.  A partir del práctico n°1 pudieron observar algunas clases que se encuentran desbalanceadas en el dataset. Pensando en el práctico siguiente y la variable target para predecir la asistencia de turnos médicos (attendance), ¿Consideran que la misma se encuentra desbalanceada? ¿Creen que esto podría afectar de alguna manera la predicción? En el caso de estar desbalanceada, ¿Tratarían de aplicar alguna técnica de balanceo? De ser positiva la respuesta se les invita a explorar las diversas técnicas de balanceo de datos y elegir la que consideren más adecuada para este caso justificando su respuesta. (sólo dejar en el práctico la técnica elegida)
11.   Para trabajar con datos sensibles y confidenciales es de gran importancia utilizar técnicas de despersonalización o anonimización de datos. Si quisieran proteger los nombres de especialistas y hospitales, ¿qué técnicas se les ocurren? Pueden mencionar varias, pero es de gran importancia que pueden aplicar al menos una sobre el dataset (es solo a fines de poner en práctica lo aprendido en clase, luego ese dataset anonimizado no será utilizado para el práctico 3)
12.   Guardar el dataset resultante con un nuevo nombre (sin la técnica de anonimización aplicada en el punto 11) para trabajarlo más adelante.

# Se evaluarán los siguientes aspectos:
  
1- Que se apliquen los conceptos vistos con los profes en el teórico y en el práctico.

2- Que el entregable no sea solo la notebook. El informe debe tener un mensaje claro y debe presentarse en un formato legible para cualquier tipo de stakeholder. Además, se deberá entregar el dataset trabajado.

3- Capacidad de Análisis y justificación. Proactividad en la profundización de los análisis.

4- Criterio para elegir que solución aplicar en cada caso y con qué método implementarla.

5- Calidad y estilo de código.


Deadline tentativo para la entrega 29/07/2024

### **Cargando el dataset**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import missingno as msno
%matplotlib inline

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/kunan-sa/mentoria_2024/master/dataset/base_asistencia_anon.csv", encoding='utf8')
df=df.drop(columns=['Unnamed: 0'])
df_original = df.copy()
df_original.head(3)

,doc_id,doc_full_name,msp_name,msp_id,hos_id,heq_id,hin_name,hin_id,age_avg,dni_asistance_rate,...,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,action_A_count,action_B_count,action_C_count,app_days_gap,patient_id,event_id,attendance
0,215,Carmen Sánchez,Neurologa,159,Hospital Dragonstone,3,PAMI INSTNACSERVSOCJUBY PENSPAMI,0,77.0,72.727273,...,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0
1,1855,Sofía García,CARDIOLOGIA INFANTIL,141,Instituto Harrenhal,12,OSDE,0,7.0,50.000000,...,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0
2,215,Carmen Sánchez,Neurologa,159,Hospital Dragonstone,3,PAMI INSTNACSERVSOCJUBY PENSPAMI,0,60.0,37.500000,...,71.085359,69.875045,2023-10-10 13:40:00.000,10,0,2,33,72497,320006,0


## **A ESTA VARIABLE VAMOS A REALIZARLE TODAS LAS MODIFICACIONES PERTINENTES DEL DATASET**

In [3]:
df_base=df_original.copy()

#Agregaremos la columna periodo
df_base['periodo'] = pd.to_datetime(df_base['app_start_dt'])
df_base['periodo'] = df_base['periodo'].dt.to_period('M').astype(str)

#Eliminamos msp_id y hin_name	(id propio de cada clínica, va a diferir con cada clínica este nro)
df_base=df_base.drop(['msp_id','hin_name'],axis=1)

#Renombramos columnas
df_base=df_base.rename(columns={'action_A_count':'qt_sac_pac_ac',"action_B_count":"qt_canc_pac_ac",'action_C_count':'qt_cons_pac'})


df_base.head(2)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0,2023-06
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0,2023-07


# **1. Verificar la información brindada por el dataset a partir del práctico n°1. ¿Hay algo que les haya llamado la atención y consideran que puede estar mal tomado el dato, o sesgado? La inclusión del mismo, ¿podría generar problemas a futuro? Tomar una decisión por cada dato o información que consideren no fiable y justificar su respuesta.**

**RESPUESTA:**
Nos llamó la atención los turnos:

* PONIENTE: Baja tasa de asistencia tanto a nivel hospital, como médicos como especialidades
* Hospitales con especialidades con dos tasas de asistencia
* Medicos con dos identificaciones e incluso por sede




In [4]:
df_uno=df_base.copy()
df_uno.head(2)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0,2023-06
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0,2023-07


# **CONSULTORIO PONIENTE**

---



In [5]:
df_uno_poniente=df_uno.copy()
df_uno_poniente=df_uno_poniente[df_uno_poniente['hos_id']=='Consultorios Poniente']
df_uno_poniente.head(3)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
14,995,Jan Levinson,TRAUMATOLOGIA,Consultorios Poniente,1,0,24.0,50.0,22.519084,22.323462,20.803002,2023-07-24 08:40:00.000,2,0,0,32,22818,416643,0,2023-07
17,939,Joe Dugan,CLINICA MEDICA,Consultorios Poniente,1,0,50.0,40.0,17.877095,17.879418,20.803002,2023-09-14 08:00:00.000,3,0,0,20,6012,428204,0,2023-09
24,945,Daenerys Targaryen,TRAUMATOLOGIA,Consultorios Poniente,1,0,17.0,0.0,22.033898,22.323462,20.803002,2023-10-06 09:40:00.000,4,0,6,37,38304,428800,0,2023-10


In [6]:
import plotly.graph_objects as go
attendance_counts = df_uno_poniente.groupby(['periodo', 'attendance']).size().unstack(fill_value=0)
attendance_counts.columns = ['Inasistencia', 'Asistencia']

fig = go.Figure(data=[
    go.Bar(name='Asistencia', x=attendance_counts.index, y=attendance_counts['Asistencia'], marker_color='green',text=attendance_counts['Asistencia']),
    go.Bar(name='Inasistencia', x=attendance_counts.index, y=attendance_counts['Inasistencia'], marker_color='red',text=attendance_counts['Inasistencia'],)])
fig.update_layout(barmode='stack', title='CONSULTORIO PONIENTE: Asistencia e Inasistencia por Mes', xaxis_title='Periodo', yaxis_title='Cantidad')

fig.show()


In [7]:
import plotly.graph_objects as go
attendance_counts = df_uno_poniente[['doc_full_name','doc_asistance_rate']].drop_duplicates()

fig = go.Figure(data=[go.Bar(name='Tasa de Asistencia', x=attendance_counts['doc_full_name'], y=attendance_counts['doc_asistance_rate'],
                             marker_color='blue', text=attendance_counts['doc_asistance_rate'].round(2), textposition='auto')])

fig.update_layout(title='Tasa de Asistencia por Médico', xaxis_title='Nombre del Médico', yaxis_title='Tasa de Asistencia (%)', xaxis={'categoryorder': 'total descending'})

fig.show()

In [8]:
import plotly.graph_objects as go
attendance_counts = df_uno_poniente[['msp_name','msp_asistance_rate']].drop_duplicates().sort_values('msp_asistance_rate',ascending=False)

fig = go.Figure(data=[go.Bar(name='Tasa de Asistencia', x=attendance_counts['msp_name'], y=attendance_counts['msp_asistance_rate'],
                             marker_color='lightgray', text=attendance_counts['msp_asistance_rate'].round(2), textposition='auto')])

fig.update_layout(title='Tasa de Asistencia por Especialidad', xaxis_title='Nombre del Especialidad', yaxis_title='Tasa de Asistencia (%)', xaxis={'categoryorder': 'total descending'})

fig.show()

In [9]:
df_check=df_uno_poniente.copy()
df_check=df_check[df_check['msp_name'].isin(['ECOCARDIOGRAMA'])]#,'ORL','OTORRINOLARINGOLOGIA'
df_check

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
5018,1055,Martha M. Masters,ECOCARDIOGRAMA,Consultorios Poniente,1,0,54.0,0.0,34.589041,NaN,20.803002,2023-06-29 10:30:00.000,1,0,0,27,12392,413455,0,2023-06
46575,1055,Martha M. Masters,ECOCARDIOGRAMA,Consultorios Poniente,1,0,34.0,0.0,34.589041,NaN,20.803002,2023-06-08 10:30:00.000,1,0,0,16,16355,412146,0,2023-06


# **HOSPITALES - ESPECIALIDADES**

---



In [10]:
df_asist_dup=df_uno.copy().groupby(['hos_id','msp_name'])['msp_asistance_rate'].nunique()
multiple_asist = df_asist_dup[df_asist_dup > 1]
multiple_asist

hos_id               msp_name      
Instituto Harrenhal  ALERGIA           2
                     CARDIOLOGIA       2
                     CLINICA MEDICA    2
                     NEUMONOLOGIA      2
                     REUMATOLOGIA      2
Name: msp_asistance_rate, dtype: int64

# **MEDICOS -ID**

---



In [11]:
# Controlamos que todos los médicos (612) tengan un solo ID (sin importar hospital y sede)
unique_docs = df_uno.copy().groupby('doc_full_name')['doc_id'].nunique()
multiple_ids = unique_docs[unique_docs > 1]

if multiple_ids.empty:
    print("Todos los médicos tienen un único ID.")
else:
    print("Los siguientes médicos tienen más de un ID:")
    print(multiple_ids)

Los siguientes médicos tienen más de un ID:
doc_full_name
Agustín González     2
Facundo Fernández    2
Franco González      2
Isabella González    2
Lautaro Martínez     2
Martina Pérez        2
Mateo Martínez       2
Matías Fernández     2
Sam Owens            2
Santiago López       2
Sofía García         3
Ted Wheeler          2
Thiago Pérez         2
Valentina López      2
Name: doc_id, dtype: int64


In [12]:
unique_ids_per_group = df_uno.copy().groupby(['doc_full_name', 'hos_id', 'heq_id'])['doc_id'].nunique() # Agrupa doc_full_name, hos_id y heq_id, y cuenta los doc_id únicos

if (unique_ids_per_group == 1).all():
    print("Todos los médicos tienen un único ID por hospital y sede.")
else:
    problematic_medics = unique_ids_per_group[unique_ids_per_group != 1]
    print("Los siguientes médicos tienen más de un ID por hospital y sede:\n")
    print(problematic_medics)

Los siguientes médicos tienen más de un ID por hospital y sede:

doc_full_name     hos_id             heq_id
Lautaro Martínez  Centro Winterfell  2         2
Name: doc_id, dtype: int64


# **2. Verificar y asegurar que no existan datos duplicados.**

In [13]:
df_dos=df_base.copy()
df_dos.head(2)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0,2023-06
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0,2023-07


In [14]:
duplicados = df_dos.duplicated()

df_duplicados = df_dos[duplicados]
df_duplicados

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo


A simple vista no se ven duplicaciones en los registros.

# **3. Verificar que no existan caracteres "raros" para los datos de las variables de especialidad (msp_name) y nombre de especialistas (doc_full_name).**

In [15]:
df_tres=df_base.copy()
df_tres.head(2)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0,2023-06
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0,2023-07


Si bien a nivel filas no existen duplicaciones, sabemos que existen errores en la base de datos en algunas columnas

# **3.1 MSP NAME**

---



In [16]:
df_tres_espec=df_tres.copy()[['hos_id','msp_name']]
df_tres_espec = df_tres_espec.groupby('hos_id')['msp_name'].unique().reset_index()
df_tres_espec

,hos_id,msp_name
0,Centro Winterfell,"[CARDIOLOGIA, GINECOLOGIA Y OBSTETRICIA, TRAUM..."
1,Centro médico Casterly Rock,"[GINECOLOGIA Y OBSTETRICIA, MEDICINA CLINICA, ..."
2,Consultorios Poniente,"[TRAUMATOLOGIA, CLINICA MEDICA, RADIOLOGIA, CA..."
3,Dorne,"[GASTROENTEROLOGIA, TRAUMATOLOGIA INFANTIL, CA..."
4,Hospital Dragonstone,"[Neurologa, Ginecologa, Espirometras, Traumato..."
5,Instituto Harrenhal,"[CARDIOLOGIA INFANTIL, PEDIATRIA, NEUMONOLOGIA..."


Acá partir de la tabla anterior, se pueden observar caracteres en minúscula para el hospital Dragonstone y palabras como 'Neurologa' 'Ginecologa' 'Traumatologa' que en otros hospitales figuran como 'Neurología', 'Ginecología' y 'Traumatología'

# **3.2 DOC FULL NAME**

---

Anteriormente en el punto 1 habíamos identificado los médicos con más de un ID

In [17]:
# @title
list_id=multiple_ids.copy()
list_id=list_id.reset_index()['doc_full_name'].to_list()
list_id

['Agustín González',
 'Facundo Fernández',
 'Franco González',
 'Isabella González',
 'Lautaro Martínez',
 'Martina Pérez',
 'Mateo Martínez',
 'Matías Fernández',
 'Sam Owens',
 'Santiago López',
 'Sofía García',
 'Ted Wheeler',
 'Thiago Pérez',
 'Valentina López']

In [18]:
# @title
df_tres_doc=df_tres.copy()[['hos_id','doc_full_name','msp_name','heq_id','doc_id']]
df_tres_doc=df_tres_doc[df_tres_doc['doc_full_name'].isin(list_id)]
df_tres_doc = df_tres_doc.groupby(['doc_full_name','hos_id','msp_name','heq_id'])['doc_id'].unique().reset_index()
df_tres_doc

,doc_full_name,hos_id,msp_name,heq_id,doc_id
0,Agustín González,Centro Winterfell,UROLOGIA,4,[2554]
1,Agustín González,Instituto Harrenhal,PEDIATRIA,14,[1901]
2,Facundo Fernández,Centro Winterfell,NUTRICION,2,[2658]
3,Facundo Fernández,Centro Winterfell,NUTRICION,4,[2658]
4,Facundo Fernández,Instituto Harrenhal,CLINICA MEDICA,10,[1032]
5,Franco González,Centro Winterfell,CIRUGIA,4,[2560]
6,Franco González,Centro Winterfell,CLINICA MEDICA,2,[951]
7,Isabella González,Centro Winterfell,GINECOLOGIA,2,[1013]
8,Isabella González,Centro Winterfell,GINECOLOGIA,3,[1702]
9,Lautaro Martínez,Centro Winterfell,ONCOLOGIA,2,[1433]


In [19]:
#Analizaremos los nombres de  especialidades de cada hospital
doc_unicos_hosp = df_tres.copy().groupby('hos_id')['doc_full_name'].unique().reset_index()
doc_unicos_hosp

,hos_id,doc_full_name
0,Centro Winterfell,"[Mary Augustine, Laura Ruiz, Susan Bones, ..."
1,Centro médico Casterly Rock,"[ Arizona Robbins, Pete Wilder, Lily Scott Pa..."
2,Consultorios Poniente,"[Jan Levinson, Joe Dugan, Daenerys Targaryen, ..."
3,Dorne,"[ Marcus Belby, Teddy Altman, Jaqen H'ghar,..."
4,Hospital Dragonstone,"[Carmen Sánchez, Graham Pritchard, Alex K..."
5,Instituto Harrenhal,"[Sofía García, Matt Davis, Vincent Crabbe,..."


Algunos médicos trabajan en dos hospitales distintos,  
* Isabella González y Facundo Fernández	tiene distintos ID para la misma especialidad en dos sedes
* Lautaro Martínez tiene un ID por cada especialidad
*


# **4. A partir de los valores faltantes reconocidos en el práctico N° 1 se deberán proponer dos soluciones posibles para trabajar con estos datos. En este punto deberán ejecutar ambas soluciones y fundamentar su aplicación. Además, deberán elegir una para obtener su dataset final, explicando su elección.**

En base al análisis del TP1, las columnas que tienen valores faltantes en sus datos son: dni_asistance_rate, doc_asistance_rate, msp_asistance_rate

In [20]:
df_cuatro=df_base.copy()
df_cuatro.head(2)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0,2023-06
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0,2023-07


In [21]:
valores_faltantes = df_cuatro.isnull().sum()
valores_faltantes

doc_id                    0
doc_full_name             0
msp_name                  0
hos_id                    0
heq_id                    0
hin_id                    0
age_avg                   0
dni_asistance_rate       54
doc_asistance_rate        3
msp_asistance_rate        2
hosheq_asistance_rate     0
app_start_dt              0
qt_sac_pac_ac             0
qt_canc_pac_ac            0
qt_cons_pac               0
app_days_gap              0
patient_id                0
event_id                  0
attendance                0
periodo                   0
dtype: int64

In [22]:
df_cuatro.dtypes

doc_id                     int64
doc_full_name             object
msp_name                  object
hos_id                    object
heq_id                     int64
hin_id                     int64
age_avg                  float64
dni_asistance_rate       float64
doc_asistance_rate       float64
msp_asistance_rate       float64
hosheq_asistance_rate    float64
app_start_dt              object
qt_sac_pac_ac              int64
qt_canc_pac_ac             int64
qt_cons_pac                int64
app_days_gap               int64
patient_id                 int64
event_id                   int64
attendance                 int64
periodo                   object
dtype: object

In [23]:
#Panorama rápido del dataset
df_cuatro.describe(include='all')

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
count,47088.000000,47088,47088,47088,47088.000000,47088.000000,47088.000000,47034.000000,47085.000000,47086.000000,47088.000000,47088,47088.000000,47088.000000,47088.000000,47088.000000,47088.000000,4.708800e+04,47088.000000,47088
unique,NaN,612,107,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
top,NaN,Carmen Sánchez,TRAUMATOLOGIA,Centro Winterfell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-11 10:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07
freq,NaN,789,4212,13751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12195
mean,993.035487,NaN,NaN,NaN,4.369882,2828.434782,78.955212,69.136019,72.824886,73.432499,73.737895,NaN,7.616081,1.561374,1.410911,21.103317,272103.011319,2.067244e+06,0.682552,NaN
std,761.043398,NaN,NaN,NaN,3.905375,3861.789238,268.812266,35.545740,17.619093,16.100785,15.216013,NaN,7.544790,3.318205,3.897998,18.259299,255599.375664,1.451172e+06,0.465488,NaN
min,2.000000,NaN,NaN,NaN,1.000000,0.000000,7.000000,0.000000,0.000000,0.000000,20.803002,NaN,1.000000,0.000000,0.000000,0.000000,6.000000,2.728420e+05,0.000000,NaN
25%,130.000000,NaN,NaN,NaN,2.000000,0.000000,30.000000,50.000000,68.367347,70.071648,69.875045,NaN,3.000000,0.000000,0.000000,6.000000,55443.000000,3.317805e+05,0.000000,NaN
50%,1016.000000,NaN,NaN,NaN,3.000000,0.000000,44.000000,80.000000,76.247849,78.256071,78.480204,NaN,5.000000,0.000000,0.000000,15.000000,137476.500000,2.951014e+06,1.000000,NaN
75%,1583.000000,NaN,NaN,NaN,4.000000,8068.000000,60.000000,100.000000,83.760684,83.118866,81.196569,NaN,10.000000,2.000000,1.000000,34.000000,481387.000000,3.099168e+06,1.000000,NaN


# **4.1 Opción 1: Eliminar los datos faltantes**

---
En este caso no son muchos los datos faltantes, solo existen 3 columnas con alrededor de 60 casos sobre una base de 47.088. Lógicamente esta es la opción más fácil, aunque no corresponde eliminar por eliminar datos.


In [24]:
df_nan=df_cuatro.copy()
df_nan = df_nan.dropna(subset=['dni_asistance_rate', 'doc_asistance_rate', 'msp_asistance_rate'])
df_nan.head(2)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0,2023-06
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0,2023-07


In [25]:
valores_faltantes = df_nan.isnull().sum()
valores_faltantes

doc_id                   0
doc_full_name            0
msp_name                 0
hos_id                   0
heq_id                   0
hin_id                   0
age_avg                  0
dni_asistance_rate       0
doc_asistance_rate       0
msp_asistance_rate       0
hosheq_asistance_rate    0
app_start_dt             0
qt_sac_pac_ac            0
qt_canc_pac_ac           0
qt_cons_pac              0
app_days_gap             0
patient_id               0
event_id                 0
attendance               0
periodo                  0
dtype: int64

# **4.2 Opción 2: Rellenar los datos faltantes con alguna medida estadística**

---
Se podrían rellenar los valores faltantes de las columnas con la media o mediana de cada hospital o paciente según corresponda o si hay otro médico con un valor para la misma especialidad.


# **dni_asistance_rate, doc_asistance_rate, msp_asistance_rate**

In [106]:
df_mean=df_cuatro.copy()
df_mean.head(2)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0,2023-06
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0,2023-07


In [107]:
df_faltantes=df_mean.isnull().sum()
df_faltantes

doc_id                    0
doc_full_name             0
msp_name                  0
hos_id                    0
heq_id                    0
hin_id                    0
age_avg                   0
dni_asistance_rate       54
doc_asistance_rate        3
msp_asistance_rate        2
hosheq_asistance_rate     0
app_start_dt              0
qt_sac_pac_ac             0
qt_canc_pac_ac            0
qt_cons_pac               0
app_days_gap              0
patient_id                0
event_id                  0
attendance                0
periodo                   0
dtype: int64

In [108]:
pac_rate=df_cuatro.copy()
pac_rate=pac_rate[pac_rate['dni_asistance_rate'].isnull()]

print(pac_rate['msp_name'].unique())
print(pac_rate['hos_id'].unique())

pac_rate.head(2)

['TRAUMATOLOGIA' 'GINECOLOGIA' 'PEDIATRIA' 'NEUMONOLOGIA' 'CLINICA MEDICA'
 'CARDIOLOGIA' 'OFTALMOLOGA' 'DERMATOLOGIA' 'OFTALMOLOGIA' 'Holter'
 'UROLOGIA' 'CIRUGIA' 'GINECOLOGIA Y OBSTETRICIA' 'CARDIOLOGIA PEDIATRICA'
 'NEUROLOGIA' 'OTORRINOLARINGOLOGIA' 'ALERGIA' 'ENDOCRINOLOGIA'
 'CONTROL ANUAL MUJER']
['Dorne' 'Centro Winterfell' 'Centro médico Casterly Rock'
 'Instituto Harrenhal' 'Hospital Dragonstone']


,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
348,1557,Stanley Hudson,TRAUMATOLOGIA,Dorne,3,0,21.0,NaN,84.848485,80.566406,77.426536,2023-07-22 09:00:00.000,1,0,0,17,709634,2960322,0,2023-07
822,1016,Francisco Ramírez,GINECOLOGIA,Centro Winterfell,2,0,34.0,NaN,90.049517,81.180733,81.196569,2023-08-02 14:00:00.000,3,1,0,9,538786,2980872,1,2023-08


Podemos rellenar con la media de asistencia de los pacientes por hospital

In [109]:
mean_hosp=df_median.copy()[['hos_id','dni_asistance_rate']]
mean_hosp=mean_hosp[~mean_hosp['dni_asistance_rate'].isnull()]
mean_hosp=mean_hosp.groupby(['hos_id',],as_index=False)['dni_asistance_rate'].mean()
mean_hosp=mean_hosp.rename(columns={'dni_asistance_rate':'mean_hos_asis'})
mean_hosp

,hos_id,mean_hos_asis
0,Centro Winterfell,72.434063
1,Centro médico Casterly Rock,81.771809
2,Consultorios Poniente,8.879695
3,Dorne,77.752431
4,Hospital Dragonstone,68.508804
5,Instituto Harrenhal,75.625496


In [110]:
mean_doc=df_median.copy()[['hos_id',"heq_id",'doc_asistance_rate']]
mean_doc=mean_doc[~mean_doc['doc_asistance_rate'].isnull()]
mean_doc=mean_doc.groupby(['hos_id','heq_id'],as_index=False)['doc_asistance_rate'].mean()
mean_doc=mean_doc.rename(columns={'doc_asistance_rate':'mean_doc_asis'})
mean_doc

,hos_id,heq_id,mean_doc_asis
0,Centro Winterfell,2,79.745401
1,Centro Winterfell,3,72.386595
2,Centro Winterfell,4,78.190743
3,Centro médico Casterly Rock,2,85.245354
4,Centro médico Casterly Rock,5,81.263634
5,Consultorios Poniente,1,19.470896
6,Dorne,3,77.369469
7,Hospital Dragonstone,3,69.427539
8,Instituto Harrenhal,4,78.749156
9,Instituto Harrenhal,6,79.859384


In [111]:
mean_esp=df_median.copy()[['hos_id',"heq_id",'msp_asistance_rate']] #msp_name
mean_esp=mean_esp[~mean_esp['msp_asistance_rate'].isnull()]
mean_esp=mean_esp.groupby(['hos_id','heq_id'],as_index=False)['msp_asistance_rate'].mean()
mean_esp=mean_esp.rename(columns={'msp_asistance_rate':'mean_esp_asis'})
mean_esp

,hos_id,heq_id,mean_esp_asis
0,Centro Winterfell,2,78.888378
1,Centro Winterfell,3,76.797696
2,Centro Winterfell,4,78.841750
3,Centro médico Casterly Rock,2,84.017314
4,Centro médico Casterly Rock,5,84.432064
5,Consultorios Poniente,1,20.692403
6,Dorne,3,77.332868
7,Hospital Dragonstone,3,69.570868
8,Instituto Harrenhal,4,80.873776
9,Instituto Harrenhal,6,81.469948


In [112]:
df_mean=df_mean.copy()
df_mean=df_mean.merge(mean_hosp,how='left',on='hos_id')
df_mean=df_mean.merge(mean_doc,how='left',on=['hos_id','heq_id'])
df_mean=df_mean.merge(mean_esp,how='left',on=['hos_id','heq_id'])


df_mean['dni_asistance_rate']=np.where(df_mean['dni_asistance_rate'].isnull(),df_mean['mean_hos_asis'],df_mean['dni_asistance_rate'])
df_mean['doc_asistance_rate']=np.where(df_mean['doc_asistance_rate'].isnull(),df_mean['mean_doc_asis'],df_mean['doc_asistance_rate'])
df_mean['msp_asistance_rate']=np.where(df_mean['msp_asistance_rate'].isnull(),df_mean['mean_esp_asis'],df_mean['msp_asistance_rate'])


df_mean.head(2)

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,...,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo,mean_hos_asis,mean_doc_asis,mean_esp_asis
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,...,2,4,39,75571,290588,0,2023-06,68.508804,69.427539,69.570868
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,...,0,0,48,374820,3590242,0,2023-07,75.625496,86.026745,86.187105


# **Chequeamos que no existan faltantes**

In [113]:
df_faltantes=df_mean.isnull().sum()
df_faltantes

doc_id                   0
doc_full_name            0
msp_name                 0
hos_id                   0
heq_id                   0
hin_id                   0
age_avg                  0
dni_asistance_rate       0
doc_asistance_rate       0
msp_asistance_rate       0
hosheq_asistance_rate    0
app_start_dt             0
qt_sac_pac_ac            0
qt_canc_pac_ac           0
qt_cons_pac              0
app_days_gap             0
patient_id               0
event_id                 0
attendance               0
periodo                  0
mean_hos_asis            0
mean_doc_asis            0
mean_esp_asis            0
dtype: int64

# **df_base es nuestra nueva variable base con el relleno de valores**

In [117]:
df_base=df_mean.copy()
df_base=df_base.drop(['mean_hos_asis','mean_doc_asis','mean_esp_asis'],axis=1)

# **5. Hay algunas variables que contienen datos de texto pero no están normalizados, como por ejemplo "msp_name", donde encontramos que ginecología puede estar escrita como "GINECOLOGÍA", "ginecologia", "ginecologa", entre otras variantes. Es importante aplicar alguna técnica para unificar estos datos. Se recomienda el uso de regex.**


In [118]:
df_base.head()

,doc_id,doc_full_name,msp_name,hos_id,heq_id,hin_id,age_avg,dni_asistance_rate,doc_asistance_rate,msp_asistance_rate,hosheq_asistance_rate,app_start_dt,qt_sac_pac_ac,qt_canc_pac_ac,qt_cons_pac,app_days_gap,patient_id,event_id,attendance,periodo
0,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,77.0,72.727273,72.831978,71.085359,69.875045,2023-06-23 15:40:00.000,11,2,4,39,75571,290588,0,2023-06
1,1855,Sofía García,CARDIOLOGIA INFANTIL,Instituto Harrenhal,12,0,7.0,50.000000,71.698113,75.919732,85.898672,2023-07-18 09:00:00.000,1,0,0,48,374820,3590242,0,2023-07
2,215,Carmen Sánchez,Neurologa,Hospital Dragonstone,3,0,60.0,37.500000,72.831978,71.085359,69.875045,2023-10-10 13:40:00.000,10,0,2,33,72497,320006,0,2023-10
3,3,Arizona Robbins,GINECOLOGIA Y OBSTETRICIA,Centro médico Casterly Rock,5,0,34.0,50.000000,89.130435,86.914721,84.586035,2023-08-23 18:20:00.000,11,0,0,1,12341,597903,1,2023-08
4,43,Graham Pritchard,Ginecologa,Hospital Dragonstone,3,0,60.0,100.000000,76.271186,71.529776,69.875045,2023-07-27 16:15:00.000,4,0,0,13,86202,305566,1,2023-07


In [119]:
#Analizaremos los nombres de  especialidades de cada hospital
espec_unicas_por_hospital = df_base.groupby('hos_id')['msp_name'].unique().reset_index()
espec_unicas_por_hospital

,hos_id,msp_name
0,Centro Winterfell,"[CARDIOLOGIA, GINECOLOGIA Y OBSTETRICIA, TRAUM..."
1,Centro médico Casterly Rock,"[GINECOLOGIA Y OBSTETRICIA, MEDICINA CLINICA, ..."
2,Consultorios Poniente,"[TRAUMATOLOGIA, CLINICA MEDICA, RADIOLOGIA, CA..."
3,Dorne,"[GASTROENTEROLOGIA, TRAUMATOLOGIA INFANTIL, CA..."
4,Hospital Dragonstone,"[Neurologa, Ginecologa, Espirometras, Traumato..."
5,Instituto Harrenhal,"[CARDIOLOGIA INFANTIL, PEDIATRIA, NEUMONOLOGIA..."
